In [1]:
import torch
from model import swift_eye
model=swift_eye('./model_config.py')
all_state_dict={}
model.state_dict().keys()

/home/tongyuz/.conda/envs/swift_eye/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tongyuz/.conda/envs/swift_eye/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


odict_keys(['backbone.patch_embed.projection.weight', 'backbone.patch_embed.projection.bias', 'backbone.patch_embed.norm.weight', 'backbone.patch_embed.norm.bias', 'backbone.stages.0.blocks.0.norm1.weight', 'backbone.stages.0.blocks.0.norm1.bias', 'backbone.stages.0.blocks.0.attn.w_msa.relative_position_bias_table', 'backbone.stages.0.blocks.0.attn.w_msa.relative_position_index', 'backbone.stages.0.blocks.0.attn.w_msa.qkv.weight', 'backbone.stages.0.blocks.0.attn.w_msa.qkv.bias', 'backbone.stages.0.blocks.0.attn.w_msa.proj.weight', 'backbone.stages.0.blocks.0.attn.w_msa.proj.bias', 'backbone.stages.0.blocks.0.norm2.weight', 'backbone.stages.0.blocks.0.norm2.bias', 'backbone.stages.0.blocks.0.ffn.layers.0.0.weight', 'backbone.stages.0.blocks.0.ffn.layers.0.0.bias', 'backbone.stages.0.blocks.0.ffn.layers.1.weight', 'backbone.stages.0.blocks.0.ffn.layers.1.bias', 'backbone.stages.0.blocks.1.norm1.weight', 'backbone.stages.0.blocks.1.norm1.bias', 'backbone.stages.0.blocks.1.attn.w_msa.rela

In [2]:
#load backbone and neck
weights=torch.load('../train_backbone_and_neck/work_dir/epoch_30.pth')
backbone_state_dict={}
for k,v in weights['state_dict'].items():
    if 'backbone' in k:#load backbone 
        backbone_state_dict[k]=v
neck_state_dict={}
for k,v in weights['state_dict'].items():
    if 'neck' in k:#load neck
        neck_state_dict[k]=v

In [3]:
#load temporal fusion component
tracking_model=torch.load('../train_with_temporal_fusion_component/work_dir/epoch_29.pth')
tracking_head_state_dict={}
for k,v in tracking_model.items():
    if 'correlation_head' in k:
        tracking_head_state_dict[k]=v
    if 'tracking_head' in k:
        tracking_head_state_dict[k]=v

In [4]:
#load w/o temporal fusion component
detection_model=torch.load('../train_without_temporal_fusion_component/work_dir/epoch_24.pth')
detection_head_state_dict={}
for k,v in detection_model.items():
    if 'detection_head' in k:
        detection_head_state_dict[k]=v

In [6]:
# load unet

unet_state_dict={}
unet_model=torch.load('./unet/checkpoint_epoch15.pth')
unet_model.pop('mask_values', [0, 1])
for k,v in unet_model.items():
    new_k="unet."+k
    unet_state_dict[new_k]=v

In [7]:
all_state_dict.update(backbone_state_dict)
all_state_dict.update(neck_state_dict)
all_state_dict.update(tracking_head_state_dict)
all_state_dict.update(detection_head_state_dict)
all_state_dict.update(unet_state_dict)
model.load_state_dict(all_state_dict)
torch.save(model.state_dict(),'./model.pth')